In [ ]:
#!pip install --upgrade pip
#!pip install numpy
#!pip install pandas
#!pip install xlrd
#!pip install GEOS
#!pip install GeobricksProj4ToEPSG
#!conda install -c conda-forge cartopy --yes
#!pip install geopandas

In [ ]:
import numpy as np
import pandas
from IPython.display import display, HTML
import geopandas as gpd
import matplotlib.pyplot as plt

pandas.set_option('display.max_colwidth', -1)

basedir = '/home/idies/workspace/raddick_census/'

# category can be one of: demographics, ancestry, residence, transportation... 
#      household, income, employment, housing, qa

category = 'demographics'

vardir = basedir + 'variables/'
geodir = basedir + 'geography/'
datadir = basedir + 'data/'
errordir = basedir + 'error/'

print('Reading metadata...')
metadata_df = pandas.read_csv(vardir+'variables_acs2016_{0:}.csv'.format(category), low_memory=False, index_col=0, encoding='utf-8')
metadata_df.index.name = ''
metadata_df = metadata_df.rename(columns={'variable.1': 'variable'})

geo_metadata_df = pandas.read_csv(geodir+'geo_variables_acs2016.csv', low_memory=False, index_col=0, encoding='utf-8')

print('Reading geography...')
geo_df = pandas.read_csv(geodir+'geo_acs2016.csv', low_memory=False, index_col=0, encoding='utf-8')
geo_df = geo_df.set_index('GEOID', drop=False)

print('Reading data (estimates)...')
data_df = pandas.read_csv(datadir+'data_acs2016_{0:}.csv'.format(category), low_memory=False, index_col=0, encoding='utf-8')

print('Reading margins of error...')
error_df = pandas.read_csv(errordir+'error_acs2016_{0:}.csv'.format(category), low_memory=False, index_col=0, encoding='utf-8')

print('Calculating geography sumary levels...')
data_df['SUMLEVEL'] = data_df['GEOID'].apply(lambda x: int(x[0:3]))

print('Documenting geography summary levels...')
sumlevel_df = pandas.read_csv(geodir+'geo_summary_levels.csv', encoding='utf-8')
sumlevel_df = sumlevel_df.set_index('SUMLEVEL', drop=False)

print('Documenting state codes')
geo_df[['STATE','STUSAB']].dropna().drop_duplicates().sort_values('STATE').to_csv(geodir+'statecodes.csv', encoding='utf-8', index=False)
statecodes_df = pandas.read_csv(geodir+'statecodes.csv', encoding='utf-8')
statecodes_df['STATE'] = statecodes_df['STATE'].astype('int')
statecodes_df = statecodes_df.set_index('STATE', drop=False)

print('\n')
print('Done!')
#sumlevel_df

In [ ]:
def find_geography4(gdf, tofindlist, wantlevels = []):
    geosub_df = pandas.DataFrame()
    geolist = []

    if (len(tofindlist) == 0):
        print('CAUTION: No geography search phrases specified, returning every geography!')
    if (len(tofindlist) > 4):
        print('ERROR: list contains more than four geography search phrases, searching for only the first three')
    try:
        string1 = tofindlist[0]
    except IndexError:
        string1 = ''
    try:
        string2 = tofindlist[1]
    except IndexError:
        string2 = ''
    try:
        string3 = tofindlist[2]
    except IndexError:
        string3 = ''
    try:
        string4 = tofindlist[3]
    except IndexError:
        string4 = ''
    
    # If wantlevels not specified, search through every level
    if (len(wantlevels) == 0):
        geosub_df = gdf
    for thislevel in wantlevels: 
        if (not(np.isnan(thislevel))):
            geosub_df = pandas.concat((geosub_df,gdf[gdf['SUMLEVEL'] == thislevel]))
#            print(geosub_df.head(1))
    found_df = geosub_df[geosub_df['NAME'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x) & (string4 in x))]
    geolist = found_df['GEOID'].values.tolist()

    return geolist
    
def warn_geo_level():
    
    helpshow_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geography/geo_summary_levels.csv')
    helpshow_df = helpshow_df.set_index('SUMLEVEL')

    showme = '<table><tr><th>SUMLEVEL</th><th>Description</th><th>Count</th></tr>'
    for slvl, row in helpshow_df.iterrows():
        showme += '<tr><td>'+str(slvl)+'</td><td>'+str(row['description'])+'</td><td>'+str(row['acscount'])+'</td></tr>'
    showme += '</table>'

    return showme
    
def find_variables4(mdf, tofindlist):
    # NEED TO REWRITE TO USE REGEX INSTEAD, because now only "Male" returns men; "male" returns women b/c feMALE
    varlist = []
    if (len(tofindlist) == 0):
        print('CAUTION: No search phrases specified, returning every variable!')
    if (len(tofindlist) > 4):
        print('ERROR: list contains more than four search phrases, searching for only the first three')
    try:
        string1 = tofindlist[0]
    except IndexError:
        string1 = ''
    try:
        string2 = tofindlist[1]
    except IndexError:
        string2 = ''
    try:
        string3 = tofindlist[2]
    except IndexError:
        string3 = ''
    try:
        string4 = tofindlist[3]
    except IndexError:
        string4 = ''
            
    varlist = mdf['variable'][mdf['description'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x) & (string4 in x))].values.tolist()
    return varlist

def get_data(df, varlist = [], geolist = []):

    rdf = pandas.DataFrame()
    
    if (len(geolist) == 0):
        print('Caution: No geographies specified, will return all geographies')
        rdf = df
    else:
        for thisgeo in geolist:
            try:
                rdf = rdf.append(df.loc[thisgeo])
            except:
                print('Geography {0:} not found. Skipping.'.format(thisgeo))

    if (len(varlist) == 0):
        print('CAUTION: No variables specified, will return all variables')
    else:
        vars_that_exist = ['GEOID']
        for thisvar in varlist:
            if (thisvar in df.columns):
                vars_that_exist.append(thisvar)
            else:
                print('Variable {0:} not found. Skipping.'.format(thisvar))
        rdf = rdf[vars_that_exist]
    rdf.index.name = ''

    return rdf
    
print('Done')

In [ ]:
phrases = []
phrases.append('Total Population')
phrases.append('one race')
phrases.append('White alone')
thelist = find_variables4(metadata_df, phrases)
for x in thelist:
    print(x, metadata_df['description'].loc[x])
print('\n')

#B01001_001 SEX BY AGE for Total Population% Total:
#B02001_002 RACE for Total Population% Black or African American alone
#B02003_003 DETAILED RACE for Total Population% Population of one race:% White alone
thelist = ['B01001_001', 'B02001_002', 'B02001_003']
print(thelist)
#sumlevel_df

In [ ]:
geos = []
#wantlevels = [40]
thislevel = 500
#thislevel = 40
print('Finding matching geographies...')
#geolist = find_geography4(geo_df, geos, wantlevels)
#print('Geographies found: ',geolist)
#for y in geolist:
#    print(y, geo_df['NAME'].loc[y])


#geolist = geo_df['GEOID'][geo_df['GEOID'].apply(lambda x: (('000US' in x[2:7]) & (len(x) == 9)))].tolist()
#print(geolist)

geolist_df = geo_df[(geo_df['SUMLEVEL'] == thislevel)]
#geolist_df = geo_df[(geo_df['SUMLEVEL'] == 40) & (geo_df['GEOID'].apply(lambda x: str(x)[0:5] == '04000'))]#.values.tolist()
geolist = geolist_df['GEOID'].values.tolist()
print('Data loaded for {:,.0f} geographies!'.format(len(geolist)))

#geolist_df[['SUMLEVEL', 'STUSAB', 'STATE', 'LOGRECNO', 'NAME']][geolist_df['STUSAB'] == 'FL']
#geolist_df.groupby('STUSAB').size()

In [ ]:
#statedir = '/home/idies/workspace/raddick_census/geography/STATE/'
#statefile = statedir + 'tl_2016_us_state.shp'
#states_geo = gpd.read_file(statefile)

cddir = '/home/idies/workspace/raddick_census/geography/CD/'
cdfile = cddir + 'tl_2016_us_cd115.shp'
cd_geo = gpd.read_file(cdfile)
print('Geographies read')

cd_geo = cd_geo.rename(columns={'GEOID': 'GEOID_OLD'})
cd_geo['GEOID'] = cd_geo['GEOID_OLD'].apply(lambda x: '50000US'+x)
cd_geo = cd_geo.set_index('GEOID',drop=False)

cd_geo['STATEFP'] = pandas.to_numeric(cd_geo['STATEFP'], downcast='integer', errors='coerce')

cd_geo['undefined'] = False
#50000US09ZZ: Congressional Districts not defined
#50000US17ZZ: Congressional Districts not defined
#50000US26ZZ: Congressional Districts not defined
cd_geo.loc['50000US09ZZ','undefined'] = True
cd_geo.loc['50000US17ZZ','undefined'] = True
cd_geo.loc['50000US26ZZ','undefined'] = True

cd_geo['nonvoting'] = False
#50000US1198: DC
#50000US7298: Puerto Rico
#50000US6098: American Samoa
#50000US6698: Guam
#50000US6998: Northern Mariana Islands
#50000US7898: Virgin Islands
cd_geo.loc['50000US1198','nonvoting'] = True
cd_geo.loc['50000US7298','nonvoting'] = True
cd_geo.loc['50000US6098','nonvoting'] = True
cd_geo.loc['50000US6698','nonvoting'] = True
cd_geo.loc['50000US6998','nonvoting'] = True
cd_geo.loc['50000US7898','nonvoting'] = True

cd_geo['STUSAB'] = cd_geo.merge(statecodes_df, how='left', left_on='STATEFP', right_on='STATE')['STUSAB'].values
cd_geo['DISTRICT'] = pandas.to_numeric(cd_geo['CD115FP'], downcast='integer', errors='coerce')

cd_geo['DISTRICT'] = pandas.to_numeric(cd_geo['DISTRICT'])

#tips.loc[tips['tip'] < 2, 'tip'] *= 2
cd_geo.loc[cd_geo['DISTRICT'] == 0, 'DISTRICT'] = 1


print('Found {0:.0f} voting and {1:.0f} non-voting members for a total of {2:.0f}, plus {3:.0f} undefined districts.'
      .format(len(cd_geo[cd_geo['nonvoting'] == False]), 
             len(cd_geo[cd_geo['nonvoting'] == True]),
             len(cd_geo),
             len(cd_geo[cd_geo['undefined'] == True])
            )
     )

print('Keeping only defined districts...')
cd_geo = cd_geo[cd_geo['undefined'] == False]
cd_geo = cd_geo.drop('undefined', axis=1)

print('Keeping only voting members...')
cd_geo = cd_geo[cd_geo['nonvoting'] == False]
cd_geo = cd_geo.drop('nonvoting', axis=1)

cd_geo = cd_geo.set_index('GEOID')
cd_geo.index.name = 'GEOID'

print('ok')

#cd_geo[['undefined','nonvoting','STATEFP','CD115FP','GEOID_OLD','NAMELSAD','LSAD','CDSESSN','MTFCC','FUNCSTAT','ALAND','AWATER','INTPTLAT','INTPTLON']].head(1)

In [ ]:
### GET PERCENT BLACK AND PERCENT FOR THIS GEOGRAPHY...
print('Retrieving data...')
ourdata_df = get_data(data_df, thelist, geolist)
ourdata_df = ourdata_df.join(data_df['NAME'])

ourdata_df['percent_black'] = (ourdata_df['B02001_003'] / ourdata_df['B01001_001']) * 100
ourdata_df['percent_white'] = (ourdata_df['B02001_002'] / ourdata_df['B01001_001']) * 100

print('Calculating geometries...')
# For congressional districts...
#cd_geo = cd_geo.rename(columns={'GEOID': 'GEOID_OLD'})
#cd_geo['GEOID'] = cd_geo['GEOID_OLD'].apply(lambda x: '50000US'+x)
#cd_geo = cd_geo.set_index('GEOID', drop=False)
#cd_geo = cd_geo.set_index('GEOID')

#cd_geo = cd_geo.drop('GEOID_OLD', axis=1)

#ourdata_df = ourdata_df.join(cd_geo)

thecrs = {'init': 'epsg:4326'}
#ourdata_gdf = gpd.GeoDataFrame(ourdata_df, crs=thecrs, geometry=ourdata_gdf['geometry'])
ourdata_gdf = gpd.GeoDataFrame(ourdata_df, crs=thecrs, geometry=cd_geo['geometry'])

ourdata_gdf = ourdata_gdf.set_index('GEOID', drop=False)
ourdata_gdf.index.name = 'GEOID'

ourdata_gdf = ourdata_gdf.join(cd_geo[['STUSAB', 'DISTRICT']])

maxval_white = ourdata_gdf['percent_white'].max()
maxval_black = ourdata_gdf['percent_black'].max()

#ourdata_df.sample(1)
print('Maximum percent white: {0:.1f}%; Maximum percent black: {1:.1f}%.'.format(maxval_white, maxval_black))

print('ok')

In [ ]:
#### DISPLAY FOR ONE STATE AT A TIME

showstate = 'Alabama'

ourdata_gdf = ourdata_gdf_bk

ourdata_gdf = ourdata_gdf[ourdata_gdf['NAME'].apply(lambda x: showstate in x)]

ourdata_gdf[['GEOID','NAME']]

presidential_voting = {'50000US0101': [103364, 192634],
                       '50000US0102': [94299, 185505],
                       '50000US0103': [93300, 188477],
                       '50000US0104': [50722, 233661],
                       '50000US0105': [97159, 200571],
                       '50000US0106': [86116, 233493],
                       '50000US0107': [204586, 83915]}

presidential_voting_df = pandas.DataFrame.from_dict(data=presidential_voting, orient='index')
presidential_voting_df.columns = ['clinton', 'trump']
presidential_voting_df['total'] = presidential_voting_df['clinton'] + presidential_voting_df['trump']

presidential_voting_df['percent_clinton'] = (presidential_voting_df['clinton'] / presidential_voting_df['total']) * 100
presidential_voting_df['percent_trump'] = (presidential_voting_df['trump'] / presidential_voting_df['total']) * 100
    

presidential_voting_df.index.name = 'GEOID'
presidential_voting_df

ourdata_gdf = ourdata_gdf.join(presidential_voting_df)

ourdata_gdf = ourdata_gdf.to_crs({'init': 'epsg:26930'})

#### DISPLAY BLACK AND WHITE ON TWO SEPARATE MAPS; ALSO SET COORD LIMITS MANUALLY

fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(15,12.5))

ax1 = ourdata_gdf.plot(ax=ax1, column='percent_black', cmap='Greys', edgecolor='black')
#ax1.set_title('Percent \'one race, black\' (ACS 2016)', fontsize=12)
ax1.set_aspect('equal')

ax2 = ourdata_gdf.plot(ax=ax2, column='percent_trump', cmap='seismic', edgecolor='black')
#ax2.set_title('Percent vote for Trump', fontsize=12)
ax2.set_aspect('equal')

ax1.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')
ax2.tick_params(axis='both', which='both', bottom='off', left='off', labelleft='off', labelbottom='off')

# add colorbar
cax1 = fig.add_axes([0.12, 0.08, 0.36, 0.03])
sm1 = plt.cm.ScalarMappable(cmap='Greys', norm=plt.Normalize(vmin=0, vmax=100))

# fake up the array of the scalar mappable. Urgh...
sm1._A = []

cbar1 = fig.colorbar(sm1, cax=cax1, format='%.0f', ticks=np.arange(0, 110, 10), orientation='horizontal')

cax1.tick_params(labelsize=12)
cbar1.set_label('Percent Black (2016 ACS)', fontsize=16)

# add colorbar
cax2 = fig.add_axes([0.545, 0.08, 0.36, 0.03])
sm2 = plt.cm.ScalarMappable(cmap='seismic', norm=plt.Normalize(vmin=0, vmax=100))

# fake up the array of the scalar mappable. Urgh...
sm2._A = []

cbar2 = fig.colorbar(sm2, cax=cax2, format='%.0f', ticks=np.arange(0, 110, 10), orientation='horizontal')

cax2.tick_params(labelsize=12)
cbar2.set_label('Percent vote for Trump', fontsize=16)

plt.title('Alabama 2016 Presidential election results by congressional district', fontsize=20, position=(-0.05,27.5))
plt.show()
#print('ok')
#gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
#type(ourdata_gdf)

In [ ]:
#### DISPLAY FOR THE WHOLE U.S.
#### DISPLAY BLACK AND WHITE ON TWO SEPARATE MAPS; ALSO SET COORD LIMITS MANUALLY

fig, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True, figsize=(15,12.5))

# Fine-tune figure; make subplots close to each other and hide x ticks for
# all but bottom plot.
fig.subplots_adjust(hspace=0.1)

#plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=False)

ax1 = ourdata_gdf.plot(ax=ax1, column='percent_white', cmap='viridis', edgecolor='white')
ax1.set_title('Percent of population self-reporting as \'one race, white\'', fontsize=20)

ax2 = ourdata_gdf.plot(ax=ax2, column='percent_black', cmap='viridis', edgecolor='white')
ax2.set_title('Percent of population self-reporting as \'one race, black\'', fontsize=20)

ax1.set_xlim(-130, -65)
ax1.set_ylim(23,50)

ax2.set_xlim(-130, -65)
ax2.set_ylim(23,50)

ax1.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    left='off',
    right='off',
    labelleft='off',
    labelbottom='off') # labels along the bottom edge are off

ax2.tick_params(
    axis='both',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    left='off',
    right='off',
    labelleft='off',
    labelbottom='off') # labels along the bottom edge are off


# add colorbar
cax1 = fig.add_axes([0.9, 0.53, 0.03, 0.37])
sm1 = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=maxval_white))

# fake up the array of the scalar mappable. Urgh...
sm1._A = []

cbar1 = fig.colorbar(sm1, cax=cax1, format='%.0f', ticks=np.arange(0, 100, 10))

cax1.tick_params(labelsize=12)
cbar1.set_label('Percent', fontsize=16)

# add colorbar
cax2 = fig.add_axes([0.9, 0.125, 0.03, 0.36875])
sm2 = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=maxval_black))

# fake up the array of the scalar mappable. Urgh...
sm2._A = []

cbar2 = fig.colorbar(sm2, cax=cax2, format='%.0f', ticks=np.arange(0, 70, 10))

cax2.tick_params(labelsize=12)
cbar2.set_label('Percent', fontsize=16)

plt.show()
#print('ok')
#gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
#type(ourdata_gdf)

In [ ]:
### EXAMPLE: SIMPLE PLOT OF EARTH COUNTRIES
#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world.plot()

### EXAMPLE: PLOT ALL COUNTRIES THAT EXTEND INTO THE SOUTHERN HEMPISPHERE
#southern_world = world.cx[:, :0]
#southern_world.plot(figsize=(10, 3))
#plt.show()


### EXAMPLE: OVERLAY CITIES ON WORLD MAP
#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
#cities = cities.to_crs(world.crs)
#base = world.plot(color='white', edgecolor='black', figsize=(12,12))
#cities.plot(ax=base, marker='o', color='red', markersize=5, figsize=(12,12))
#plt.show()

### EXAMPLE: COLOR-CODE COUNTRIES BY PER CAPITA GDP
#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world = world[(world.pop_est>0) & (world.name!="Antarctica")]
#world['gdp_per_cap'] = world.gdp_md_est / world.pop_est
#world.plot(column='gdp_per_cap', figsize=(12,12), cmap='OrRd')
#plt.show()


### EXAMPLE: HELLO ED
import numpy as np
import string
from shapely.geometry import Point
import matplotlib.pyplot as plt
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.set_index('iso_a3')

world['ed'] = np.where(world['name'].apply(lambda x: 'ed' in x.lower()),1,0)
world['ed']
raw_data = [("Ed", 39.299236, -76.609383)]

edplace = pandas.DataFrame(raw_data, columns=["name", "latitude", "longitude"])
edplace['geometry'] = [Point(xy) for xy in zip(edplace.longitude, edplace.latitude)]

thecrs = {'init': 'epsg:4326'}
edplacer = gpd.GeoDataFrame(edplace, crs=thecrs, geometry="geometry")
edplacer = edplacer.to_crs(world.crs)

fig, ax = plt.subplots(figsize=(16,10))
ax.set_aspect('equal')
ax1 = world.plot(ax=ax, color='white', edgecolor='black')
ax2 = world[world['ed'] == 1].plot(ax=ax, color='yellow')
ax3 = edplacer.plot(ax=ax, marker='o', color='red', markersize=32)
ax.annotate('ED', xy=(-76.609383, 39.299236), xytext=(-42.5, 35),
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=22
            )
plt.xlim(-180, 180)
plt.ylim(-90,90)
plt.title('hello ed', fontsize=28)

plt.xticks(np.arange(-180, 180.01, 45), fontsize=18)
plt.xlabel('Longitude', fontsize=20)
plt.yticks(np.arange(-90, 90.01, 30), fontsize=18)
plt.ylabel('Latitude', fontsize=20)
yellow_patch = mpatches.Patch(color='yellow', label="Country has 'Ed' in its name (e.g. swEDen)")
plt.legend(handles=[yellow_patch], loc='lower right', fontsize=16)

plt.show()
#print('ok')
#sumlevel_df

In [ ]:
#### CARTOPY EXAMPLES


import cartopy.crs as ccrs
import matplotlib.pyplot as plt

#ax = plt.axes(projection=ccrs.PlateCarree())
#ax.coastlines()

#plt.show()

#ax = plt.axes(projection=ccrs.Mollweide())
#ax.stock_img()
#ax.coastlines()
#plt.show()

ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
ax.coastlines()

ny_lon, ny_lat = -75, 43
delhi_lon, delhi_lat = 77.23, 28.61

plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='blue', linewidth=2, marker='o',
         transform=ccrs.Geodetic(),
         )

plt.plot([ny_lon, delhi_lon], [ny_lat, delhi_lat],
         color='gray', linestyle='--',
         transform=ccrs.PlateCarree(),
         )

plt.text(ny_lon - 3, ny_lat - 12, 'New York',
         horizontalalignment='right',
         transform=ccrs.Geodetic())

plt.text(delhi_lon + 3, delhi_lat - 12, 'Delhi',
         horizontalalignment='left',
         transform=ccrs.Geodetic())

plt.show()

In [ ]:
#POPULATION BY CENSUS REGION
data_df['SUMLEVEL'] = data_df['GEOID'].apply(lambda x: int(x[0:3]))

print('Estimated U.S. population: {0:,.0f}'.format(data_df['B01001_001'].loc['01000US']))

print('For the four census regions: {0:,.0f}'.format(data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US'].sum()))
print('\n')
print('Population by census region:')
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US1'], data_df['B01001_001'].loc['02000US1']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US2'], data_df['B01001_001'].loc['02000US2']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US3'], data_df['B01001_001'].loc['02000US3']))
print('{0:} {1:,.0f}'.format(data_df['NAME'].loc['02000US4'], data_df['B01001_001'].loc['02000US4']))

#print('Men: {0:,.0f} (error: {1:,.0f})'.format(data_df['B01001_002'].loc['01000US'], error_df['B01001_002'].loc['01000US']))
#print('Women: {0:,.0f} (error: {1:,.0f})'.format(data_df['B01001_026'].loc['01000US'], error_df['B01001_026'].loc['01000US']))

#print('Margin of error: {0:,.0f}'.format(error_df['B01001_001'].loc['01000US']))

In [ ]:
# POPULATION BY STATE
data_df[data_df['SUMLEVEL'] == 40]

display(HTML('<h2>Population by state</h2>'))

statetable = '<table><tr><th>State</th><th>Population</th></tr>'
for thisgeoid, thisrow in data_df[data_df['GEOID'].apply(lambda x: x[0:5]) == '04000'].iterrows():
    statetable += '<tr>'
    statetable += '<td>{0:}</td><td style="text-align:right;padding:5px">{1:,.0f}</td>'.format(thisrow['NAME'], thisrow['B01001_001'])
    statetable += '</tr>'
statetable += '</table>'

display(HTML(statetable))

total_population = data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:5]) == '04000'].sum()

display(HTML(('TOTAL of all 50 states: <b>{0:,.0f}</b> (not equal to national b/c military etc. is federal)'.format(total_population))))
#print('For the four census regions: {0:,.0f}'.format(data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US'].sum()))
#data_df['B01001_001'][data_df['GEOID'].apply(lambda x: x[0:7]) == '02000US']